In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=dae1f6c16c6cc2e5ed05a88f8926466e999c6b27e40a037db797c85fa8a13421
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F15 '''
def F15(X):
    f = bn.F15()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.32 ,random_state=0 , l1_ratio=0.9795, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 37.213462 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_15_4000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_15_800Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [5]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,201):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 200, [5] * 200 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 200000)

n_obs , dim =  30, 200
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([10,600])
for i in range(0, 10):
  print ('Run : '+ str(i))
  min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
  X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(9_w,19)-aCMA-ES (mu_w=5.6,w_1=29%) in dimension 200 (seed=213026, Fri Apr 17 20:32:46 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     19 1.750167043633471e+04 1.0e+00 2.44e+00  2e+00  2e+00 0:00.1
    2     38 1.635299445945508e+04 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [17619.85842897] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [16762.70103571] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    3     57 1.482429217939609e+04 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3
   31    589 5.702828683646718e+03 1.1e+00 1.97e+00  2e+00  2e+00 0:03.4
   70   1330 5.410222649238640e+02 1.1e+00 1.59e+00  2e+00  2e+00 0:07.5
  100   1900 -5.001716024735244e+03 1.2e+00 1.25e+00  1e+00  1e+00 0:10.6
  158   3002 -1.589845512437425e+04 1.2e+00 7.68e-01  8e-01  8e-01 0:16.7
  200   3800 -2.380554164520953e+04 1.2e+00 4.86e-01  5e-01  5e-01 0:21.1
  276   5244 -2.858578565739321e+04 1.2e+00 2.04e-01  2e-01  2e-01 0:29.1
  300   5700 -2.912194318169776e+04 1.2e+00 1.53e-01  2e-01  2e-01 0:31.7
  396   7524 -2.969847199342889e+04 1.2e+00 5.25e-02  5e-02  5e-02 0:41.7
  400   7600 -2.970642667141828e+04 1.2e+00 5.05e-02  5e-02  5e-02 0:42.2
  500   9500 -2.976694453091886e+04 1.2e+00 1.78e-02  2e-02  2e-02 0:52.4
  600  11400 -2.977509110866268e+04 1.2e+00 5.93e-03  6e-03  6e-03 1:03.1
  700  13300 -2.977593606268254e+04 1.2e+00 1.94e-03  2e-03  2e-03 1:13.4
  800  15200 -2.977604250353733e+04 1.2e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [20084.57006765] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [19557.24928367] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 1.557686581481732e+04 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2
    3     57 1.514248022553358e+04 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


...

    3     57 1.636391991804822e+04 1.0e+00 2.33e+00  2e+00  2e+00 0:00.3
   30    570 4.845112927556191e+03 1.1e+00 1.89e+00  2e+00  2e+00 0:03.3
   68   1292 1.649329247571794e+03 1.2e+00 1.73e+00  2e+00  2e+00 0:07.4
  100   1900 -2.432290742483579e+03 1.2e+00 1.40e+00  1e+00  1e+00 0:10.9
  155   2945 -1.116240478540994e+04 1.2e+00 8.96e-01  9e-01  9e-01 0:17.0
  200   3800 -2.159772322018317e+04 1.2e+00 5.42e-01  5e-01  5e-01 0:21.9
  273   5187 -2.773059905966866e+04 1.2e+00 2.52e-01  3e-01  3e-01 0:29.9
  300   5700 -2.879091320398894e+04 1.2e+00 1.85e-01  2e-01  2e-01 0:32.9
  394   7486 -2.966354786382790e+04 1.2e+00 6.47e-02  6e-02  6e-02 0:43.0
  400   7600 -2.967954752335240e+04 1.2e+00 5.97e-02  6e-02  6e-02 0:43.7
  500   9500 -2.976295591132963e+04 1.2e+00 1.97e-02  2e-02  2e-02 0:54.6
  600  11400 -2.977480662082273e+04 1.2e+00 6.78e-03  7e-03  7e-03 1:05.3
  700  13300 -2.977592376928247e+04 1.2e+00 2.17e-03  2e-03  2e-03 1:16.0
  800  15200 -2.977604216083260e+04 1.3e+

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [20084.57006765] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [21989.00234606] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


    2     38 1.804511059575311e+04 1.0e+00 2.38e+00  2e+00  2e+00 0:00.2
    3     57 1.800659380397243e+04 1.0e+00 2.34e+00  2e+00  2e+00 0:00.3


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:347: UserWarning: function_values is not a list of scalars,
                        the first element equals [19899.72254505] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


   33    627 4.568049805061966e+03 1.1e+00 2.01e+00  2e+00  2e+00 0:03.3
   73   1387 -2.056995000160276e+03 1.2e+00 1.67e+00  2e+00  2e+00 0:07.4
  100   1900 -6.419599665787619e+03 1.2e+00 1.38e+00  1e+00  1e+00 0:10.0
  160   3040 -1.167129683931367e+04 1.2e+00 9.08e-01  9e-01  9e-01 0:16.0
  200   3800 -1.515351920302343e+04 1.2e+00 6.44e-01  6e-01  6e-01 0:20.0
  279   5301 -2.016652606288518e+04 1.2e+00 4.16e-01  4e-01  4e-01 0:28.1
  300   5700 -2.130144595626932e+04 1.3e+00 3.69e-01  4e-01  4e-01 0:30.2
  400   7600 -2.386395523340202e+04 1.3e+00 1.69e-01  2e-01  2e-01 0:40.2
  500   9500 -2.463991160991772e+04 1.4e+00 9.45e-02  9e-02  1e-01 0:50.3
  600  11400 -2.517745676875261e+04 1.5e+00 8.52e-02  8e-02  9e-02 1:00.4
  700  13300 -2.585551406554814e+04 2.0e+00 1.04e-01  1e-01  1e-01 1:10.4
  800  15200 -2.709772470972281e+04 2.5e+00 1.41e-01  1e-01  2e-01 1:20.5
  900  17100 -2.766113527438693e+04 2.6e+00 8.56e-02  8e-02  1e-01 1:30.5
 1000  19000 -2.794936217987501e+04 2.6

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F15_X_Values_first.csv')
Krig_Fun = np.zeros(10)
SVM_Fun = np.zeros(10)
ELN_Fun = np.zeros(10)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F15(X_Values.iloc[i,:200])
    SVM_Fun [i] = F15(X_Values.iloc[i,200:400])
    ELN_Fun [i] = F15(X_Values.iloc[i,400:600])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([3719.67603761]), count=array([1])) 0.00011724037957600895
SVM
ModeResult(mode=array([18245.74669736]), count=array([1])) 3660.171248295982
ELN
ModeResult(mode=array([19281.51654953]), count=array([1])) 19341.258928723524
